## 1. Initialize environment
Set up core libraries, plotting defaults, and project directory paths required for the model zoo experiments.

In [ ]:
# Standard Library
import sys
import warnings
from pathlib import Path

# Data manipulation
import numpy as np
import pandas as pd

# Time series modeling
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoETS,
    AutoTheta,
    CrostonOptimized,
    HoltWinters,
    Naive,
    SeasonalNaive,
    WindowAverage,
)

# Feature engineering
import tsforge as tsf

# Visualization
import matplotlib.pyplot as plt

# --- Path Configuration ---
MODULE_DIR = Path().resolve()
PROJECT_ROOT = MODULE_DIR.parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

# --- Local Imports ---
from forecast_foundations import (
    CacheManager,
    ArtifactManager,
    get_notebook_name,
)

# --- Settings ---
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")

# --- Paths ---
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# --- Managers ---
NB_NAME = get_notebook_name()
cache = CacheManager(PROJECT_ROOT / ".cache" / NB_NAME)
artifacts = ArtifactManager(PROJECT_ROOT / "artifacts")

## 2. Define baseline model zoo
List the classical forecasting models and shared column names used throughout the notebook.

In [20]:
MODEL_ZOO = [
    Naive(),
    CrostonOptimized(),
    WindowAverage(window_size=4, alias="MA4"),  # often a strong baseline in retail forecasting
    SeasonalNaive(season_length=52, alias="SN52"),  # predict last years value
    HoltWinters(season_length=52, alias="HW52"),  # holtwinters is a special, simplified case of ETS
    AutoTheta(
        season_length=52,
        decomposition_type="multiplicative",
    ),  # automatic selection of the best Theta model/method, use multiplicative seasonlity
    AutoETS(
        season_length=52,
        model="ZZZ",
        damped=True,  # dampen the trend
        phi=0.98,
    ),  # AutoETS (Error,Trend,Seasonal) is a generalized State-Space Exponential Smoothing that automatically selects the best model
]

ID_COL = "unique_id"
TIME_COL = "ds"
TARGET_COL = "y"

## 3. Load M5 weekly dataset
Pull the preprocessed M5 parquet with demand history for all series.

In [ ]:
m5_dataset = pd.read_parquet(PROJECT_ROOT / "artifacts" / "01_foundations" / "output" / "1.08.parquet")

#slice relevant columns 
m5_dataset = m5_dataset[['unique_id', 'ds', 'y', 'item_id']]


m5_dataset.head()

In [22]:
# prepare statsforecast class

stat_fcst = StatsForecast(
    models=MODEL_ZOO,
    freq="W-SUN",  # set frequency to weekly
    n_jobs=-1,  # use all cpu cores
    fallback_model=Naive(),  # if we have any models that fail, fall back to naive
)


## 4. Initialize StatsForecast engine
Configure parallel forecasting with all models, weekly frequency, and fallback strategy.

## 5. Fit model zoo and produce horizon predictions
Run parallel fit on every series and obtain 13-step forecasts from each baseline model.

In [23]:
# fit entire model zoo in parallel! 
stat_fcst.fit(
    df = m5_dataset,
    id_col=ID_COL,
    time_col=TIME_COL,
    target_col=TARGET_COL
)

# predict future timesteps with all models for all series! 
preds = stat_fcst.predict(h=13)

/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packag

## 6. Inspect fitted model objects
Capture the fitted model metadata returned by StatsForecast for later diagnostics.

In [24]:
fitted_models = stat_fcst.fitted_ # returns a numpy array of mxn fitted models 

# inspect what type of theta model we have... this is just a single thing you can do with a fitted object 
fitted_models[:,5][0].model_['modeltype']

'OTM'

## 7. Convert fitted models to tidy format
Transform the array of fitted model objects into a long-form DataFrame for downstream analysis.

In [25]:
fitted_model_df = pd.DataFrame(
    data = fitted_models,
    index=stat_fcst.uids,
).reset_index().melt(
    id_vars='unique_id',
    value_name='model',
).drop("variable",axis=1)

## 8. Run fast forecasting pass
Invoke StatsForecast's optimized `forecast` method to produce 13-step predictions for every series.

In [26]:
# now lets just use the .forecast method to fit/predict as fast as possible 

forecast = stat_fcst.forecast(
    h=13,
    df=m5_dataset,
    id_col=ID_COL,
    target_col=TARGET_COL,
    time_col=TIME_COL,
)

/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/statsforecast/ets.py:653: RuntimeWarning: divide by zero encountered in scalar divide
  sigma2 = np.sum(e**2) / (ny - np_ - 1)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packag

## 9. Visualize AutoETS forecasts
Plot representative time series with AutoETS overlays to gauge face validity of the generated predictions.

In [ ]:
tsf.plot_timeseries(
    df = m5_dataset,
    id_col=ID_COL,
    date_col=TIME_COL,
    value_col=TARGET_COL,
    group_col=ID_COL,
    freq="W-SAT",
    max_ids=4,
    mode='facet',
    forecast=forecast,
    forecast_value_col="AutoETS",
    wrap=2,

)